# DSA 2040A Lab 5: ETL Load Phase - SQLite Implementation

## Objective
Load transformed datasets into SQLite database and verify the loading process.

## Data Sources
- `transformed_full.csv` - Full transformed dataset
- `transformed_incremental.csv` - Incremental transformed dataset

## Output Format
- SQLite database file: `loaded/full_data.db`

## 1. Import Required Libraries

In [1]:
import pandas as pd
import sqlite3
import os
from datetime import datetime

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"Current working directory: {os.getcwd()}")

Libraries imported successfully!
Pandas version: 2.3.2
Current working directory: c:\Users\Admin\Desktop\MY DATAWARE HOUSE WORK\loaddata\loaddata


## 2. Load Transformed Datasets

In [2]:
# Load the transformed datasets
print("Loading transformed datasets...")

try:
    df_full = pd.read_csv('transformed_full.csv')
    df_incremental = pd.read_csv('transformed_incremental.csv')
    
    print(f" Full dataset loaded: {df_full.shape[0]} rows, {df_full.shape[1]} columns")
    print(f" Incremental dataset loaded: {df_incremental.shape[0]} rows, {df_incremental.shape[1]} columns")
    
except FileNotFoundError as e:
    print(f" Error loading datasets: {e}")
    raise

Loading transformed datasets...
 Full dataset loaded: 10002 rows, 25 columns
 Incremental dataset loaded: 1500 rows, 25 columns


## 3. Data Overview

In [3]:
# Display basic information about the datasets
print(" FULL DATASET OVERVIEW ")
print(f"Shape: {df_full.shape}")
print(f"Columns: {list(df_full.columns)}")
print("\nFirst 3 rows:")
display(df_full.head(3))

print("\n INCREMENTAL DATASET OVERVIEW ")
print(f"Shape: {df_incremental.shape}")
print("\nFirst 3 rows:")
display(df_incremental.head(3))

 FULL DATASET OVERVIEW 
Shape: (10002, 25)
Columns: ['customer_id', 'product', 'category', 'quantity', 'unit_price', 'order_date', 'region', 'payment_method', 'total_cost', 'order_year', 'order_month', 'order_quarter', 'order_day_of_week', 'is_weekend', 'days_since_epoch', 'customer_transaction_number', 'is_quantity_outlier', 'is_price_outlier', 'is_extreme_outlier', 'price_tier', 'order_size_category', 'revenue_tier', 'customer_type', 'season', 'product_group']

First 3 rows:


,customer_id,product,category,quantity,unit_price,order_date,region,payment_method,total_cost,order_year,...,customer_transaction_number,is_quantity_outlier,is_price_outlier,is_extreme_outlier,price_tier,order_size_category,revenue_tier,customer_type,season,product_group
0,CUST_10003,Textbook,Books,1,46.73,6/27/2024,Latin America,Cash,46.73,2024,...,1,False,False,False,Budget,Single Item,Low Value,New Customer,Summer,Lifestyle & Food
1,CUST_10023,Perfume,Health & Beauty,1,92.71,9/10/2024,Europe,Bank Transfer,92.71,2024,...,1,False,False,False,Mid-Range,Single Item,Low Value,New Customer,Fall,Fashion & Beauty
2,CUST_10059,Printer Paper,Office Supplies,1,23.91,11/15/2024,Asia Pacific,Credit Card,23.91,2024,...,1,False,False,False,Budget,Single Item,Low Value,New Customer,Fall,Home & Office



 INCREMENTAL DATASET OVERVIEW 
Shape: (1500, 25)

First 3 rows:


,customer_id,product,category,quantity,unit_price,order_date,region,payment_method,total_cost,order_year,...,customer_transaction_number,is_quantity_outlier,is_price_outlier,is_extreme_outlier,price_tier,order_size_category,revenue_tier,customer_type,season,product_group
0,CUST_9948,Pen Set,Office Supplies,1,40.73,10/22/2025,Middle East,Bank Transfer,40.73,2025,...,1,False,False,False,Budget,Single Item,Low Value,New Customer,Fall,Home & Office
1,CUST_22406,Cookbook,Books,3,36.72,10/22/2025,Latin America,Bank Transfer,110.16,2025,...,1,False,False,False,Budget,Small Order,Medium Value,New Customer,Fall,Lifestyle & Food
2,CUST_31118,Lamp,Home & Garden,3,503.01,10/22/2025,Asia Pacific,Paypal,1509.03,2025,...,1,False,False,False,Luxury,Small Order,Very High Value,New Customer,Fall,Home & Office


## 4. Load Data to SQLite Database

In [4]:
# Create SQLite database and load data
print("Loading data to SQLite database...")

try:
    # Create connection to SQLite database
    conn = sqlite3.connect('loaded/full_data.db')
    
    # Load full dataset to SQLite
    df_full.to_sql('full_data', conn, if_exists='replace', index=False)
    print(f" Full dataset loaded to SQLite table 'full_data'")
    
    # Load incremental dataset to SQLite
    df_incremental.to_sql('incremental_data', conn, if_exists='replace', index=False)
    print(f" Incremental dataset loaded to SQLite table 'incremental_data'")
    
    print(f"\n Database saved as: loaded/full_data.db")
    
except Exception as e:
    print(f" Error loading to SQLite: {e}")
    raise

Loading data to SQLite database...
 Full dataset loaded to SQLite table 'full_data'
 Incremental dataset loaded to SQLite table 'incremental_data'

 Database saved as: loaded/full_data.db


## 5. Verification - SQLite Database

In [5]:
# Verify SQLite database loading
print(" SQLITE VERIFICATION ")

try:
    # Check table structure
    tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql(tables_query, conn)
    print(f"Tables in database: {list(tables['name'])}")
    
    # Verify full_data table
    print("\n--- Full Data Table ---")
    count_query = "SELECT COUNT(*) as record_count FROM full_data"
    count_result = pd.read_sql(count_query, conn)
    print(f"Record count: {count_result['record_count'].iloc[0]}")
    
    # Show sample data
    sample_query = "SELECT * FROM full_data LIMIT 5"
    sample_data = pd.read_sql(sample_query, conn)
    print("\nSample data (first 5 rows):")
    display(sample_data)
    
    # Check table schema
    schema_query = "PRAGMA table_info(full_data)"
    schema_info = pd.read_sql(schema_query, conn)
    print("\n--- Table Schema ---")
    print(f"Total columns: {len(schema_info)}")
    display(schema_info[['name', 'type']].head(10))
    
    # Verify incremental_data table
    print("\n--- Incremental Data Table ---")
    count_query_inc = "SELECT COUNT(*) as record_count FROM incremental_data"
    count_result_inc = pd.read_sql(count_query_inc, conn)
    print(f"Record count: {count_result_inc['record_count'].iloc[0]}")
    
    # Show sample incremental data
    sample_query_inc = "SELECT * FROM incremental_data LIMIT 3"
    sample_data_inc = pd.read_sql(sample_query_inc, conn)
    print("\nSample incremental data (first 3 rows):")
    display(sample_data_inc)
    
except Exception as e:
    print(f" Error verifying SQLite: {e}")
finally:
    conn.close()
    print("\n Database connection closed")

 SQLITE VERIFICATION 
Tables in database: ['full_data', 'incremental_data']

--- Full Data Table ---
Record count: 10002

Sample data (first 5 rows):


,customer_id,product,category,quantity,unit_price,order_date,region,payment_method,total_cost,order_year,...,customer_transaction_number,is_quantity_outlier,is_price_outlier,is_extreme_outlier,price_tier,order_size_category,revenue_tier,customer_type,season,product_group
0,CUST_10003,Textbook,Books,1,46.73,6/27/2024,Latin America,Cash,46.73,2024,...,1,0,0,0,Budget,Single Item,Low Value,New Customer,Summer,Lifestyle & Food
1,CUST_10023,Perfume,Health & Beauty,1,92.71,9/10/2024,Europe,Bank Transfer,92.71,2024,...,1,0,0,0,Mid-Range,Single Item,Low Value,New Customer,Fall,Fashion & Beauty
2,CUST_10059,Printer Paper,Office Supplies,1,23.91,11/15/2024,Asia Pacific,Credit Card,23.91,2024,...,1,0,0,0,Budget,Single Item,Low Value,New Customer,Fall,Home & Office
3,CUST_10073,Self-Help Book,Books,1,39.41,12/14/2023,Europe,Debit Card,39.41,2023,...,1,0,0,0,Budget,Single Item,Low Value,New Customer,Winter,Lifestyle & Food
4,CUST_10077,Tablet,Electronics,4,286.05,4/20/2024,Asia Pacific,Credit Card,1144.20,2024,...,1,0,0,0,Premium,Medium Order,Very High Value,New Customer,Spring,Technology & Auto



--- Table Schema ---
Total columns: 25


,name,type
0,customer_id,TEXT
1,product,TEXT
2,category,TEXT
3,quantity,INTEGER
4,unit_price,REAL
5,order_date,TEXT
6,region,TEXT
7,payment_method,TEXT
8,total_cost,REAL
9,order_year,INTEGER



--- Incremental Data Table ---
Record count: 1500

Sample incremental data (first 3 rows):


,customer_id,product,category,quantity,unit_price,order_date,region,payment_method,total_cost,order_year,...,customer_transaction_number,is_quantity_outlier,is_price_outlier,is_extreme_outlier,price_tier,order_size_category,revenue_tier,customer_type,season,product_group
0,CUST_9948,Pen Set,Office Supplies,1,40.73,10/22/2025,Middle East,Bank Transfer,40.73,2025,...,1,0,0,0,Budget,Single Item,Low Value,New Customer,Fall,Home & Office
1,CUST_22406,Cookbook,Books,3,36.72,10/22/2025,Latin America,Bank Transfer,110.16,2025,...,1,0,0,0,Budget,Small Order,Medium Value,New Customer,Fall,Lifestyle & Food
2,CUST_31118,Lamp,Home & Garden,3,503.01,10/22/2025,Asia Pacific,Paypal,1509.03,2025,...,1,0,0,0,Luxury,Small Order,Very High Value,New Customer,Fall,Home & Office



 Database connection closed


## 6. Data Integrity Verification

In [6]:
# Verify data integrity by reading back from database
print(" DATA INTEGRITY VERIFICATION ")

try:
    # Reconnect to database for verification
    conn = sqlite3.connect('loaded/full_data.db')
    
    # Read back the full dataset
    df_loaded = pd.read_sql('SELECT * FROM full_data', conn)
    
    # Compare original vs loaded data
    print(f"Original dataset shape: {df_full.shape}")
    print(f"Loaded dataset shape: {df_loaded.shape}")
    
    # Check if shapes match
    if df_full.shape == df_loaded.shape:
        print(" Data integrity verified: Shapes match perfectly")
    else:
        print(" Data integrity issue: Shapes don't match")
    
    # Check column names
    original_cols = set(df_full.columns)
    loaded_cols = set(df_loaded.columns)
    
    if original_cols == loaded_cols:
        print(" Column integrity verified: All column names preserved")
    else:
        print(" Column integrity issue detected")
        print(f"Missing columns: {original_cols - loaded_cols}")
        print(f"Extra columns: {loaded_cols - original_cols}")
    
    # Display data types comparison
    print("\n Data Types Verification ")
    print("Sample of loaded data types:")
    print(df_loaded.dtypes.head(10))
    
    conn.close()
    
except Exception as e:
    print(f" Error in integrity verification: {e}")

 DATA INTEGRITY VERIFICATION 
Original dataset shape: (10002, 25)
Loaded dataset shape: (10002, 25)
 Data integrity verified: Shapes match perfectly
 Column integrity verified: All column names preserved

 Data Types Verification 
Sample of loaded data types:
customer_id        object
product            object
category           object
quantity            int64
unit_price        float64
order_date         object
region             object
payment_method     object
total_cost        float64
order_year          int64
dtype: object


## 7. File System Verification

In [7]:
# Check what files were created in the loaded directory
print(" FILE SYSTEM VERIFICATION ")

loaded_dir = 'loaded'
if os.path.exists(loaded_dir):
    files = os.listdir(loaded_dir)
    print(f"Files in {loaded_dir}/ directory:")
    
    for file in files:
        file_path = os.path.join(loaded_dir, file)
        file_size = os.path.getsize(file_path)
        print(f"   {file} ({file_size:,} bytes)")
        
    print(f"\n Total files created: {len(files)}")
else:
    print(f" Directory {loaded_dir} does not exist")

 FILE SYSTEM VERIFICATION 
Files in loaded/ directory:
   full_data.db (2,179,072 bytes)

 Total files created: 1


## 8. Summary Report

In [8]:
# Generate summary report
print(" LOAD PHASE SUMMARY REPORT ")
print(f"Execution Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n DATA LOADED:")
print(f"  • Full Dataset: {df_full.shape[0]:,} records, {df_full.shape[1]} columns")
print(f"  • Incremental Dataset: {df_incremental.shape[0]:,} records, {df_incremental.shape[1]} columns")

print("\n OUTPUT FORMAT:")
print("  • SQLite Database: loaded/full_data.db")
print("    - Table: full_data")
print("    - Table: incremental_data")

print("\n VERIFICATION STATUS:")
print("  • SQLite loading: SUCCESS")
print("  • Data integrity: VERIFIED")
print("  • File creation: CONFIRMED")
print("  • Schema preservation: COMPLETE")

print("\n LOAD PHASE COMPLETED SUCCESSFULLY!")
print("\n NEXT STEPS:")
print("  1. Update README.md with load documentation")
print("  2. Commit changes to Git repository")
print("  3. Push to GitHub for submission")

 LOAD PHASE SUMMARY REPORT 
Execution Date: 2025-10-30 16:43:49

 DATA LOADED:
  • Full Dataset: 10,002 records, 25 columns
  • Incremental Dataset: 1,500 records, 25 columns

 OUTPUT FORMAT:
  • SQLite Database: loaded/full_data.db
    - Table: full_data
    - Table: incremental_data

 VERIFICATION STATUS:
  • SQLite loading: SUCCESS
  • Data integrity: VERIFIED
  • File creation: CONFIRMED
  • Schema preservation: COMPLETE

 LOAD PHASE COMPLETED SUCCESSFULLY!

 NEXT STEPS:
  1. Update README.md with load documentation
  2. Commit changes to Git repository
  3. Push to GitHub for submission
